<div align="center">
    <img src='https://media-exp1.licdn.com/dms/image/C561BAQFV4oU27rzxaQ/company-background_10000/0/1562869489836?e=2147483647&v=beta&t=OMHL6Izxqw7r8S5fn9gDQM1hNMKgo3yqMlH1F3Iwm9o' style='height:300px;'>
</div>


<center><h1 style='font-size:35px'>Exploración y Curación de Datos - Melburn Australia</h1></center>

---

**Fecha de creación: 14/05/2022**

**Autores**:
* Marilina Trevisan (marilinatrevisan@gmail.com).
* Gustavo Ariel Venchiarutti (gustavo.venchiarutti@gmail.com).
* Ani Salama (anisalama@gmail.com).
* Anahí Sulca (nanisulca@gmail.com).
* Agustín Trulli (agustintrulli@gmail.com).

**Grupo: 6**

En la primera parte el objetivo es utilizar sencias SQL envevidas en en los codigos del notebook. Vamos a realizar una analisis de las ventas inmoviliarias en la ciudad de Melburn Australia, para eso utilizamos el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot). Ademas vamos a realizar la union con la tabla de Alquires, tambien de la misma ciudad.

En la segunda parte del ejercicio vamos a realizar la mismas actividades que en el punto anterior pero directamente sobre la codificacion de pandas.

# Trabajo práctico entregable - parte 1

En esta notebook, vamos a cargar el conjunto de datos de [la compentencia Kaggle](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot) sobre estimación de precios de ventas de propiedades en Melbourne, Australia.

Utilizaremos el conjunto de datos reducido producido por [DanB](https://www.kaggle.com/dansbecker). Hemos subido una copia a un servidor de la Universidad Nacional de Córdoba para facilitar su acceso remoto.

In [2]:
import os

import matplotlib.pyplot as plt
import numpy
import pandas as pd
import io

import seaborn
seaborn.set_context('talk')

#from decouple import config
from sqlalchemy import create_engine, text
import io
import sqlite3

In [ ]:
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, 'data', 'preprocessed')

In [ ]:
import plotly
plotly.__version__
# Make sure it's 4.14.3

In [ ]:
melb_df = pd.read_csv(
    'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv')
melb_df[:3]

## Ejercicio 1 SQL: 





###Parte 1:

Crear un motor de base de datos en SQLite utilizando la libreria [SQLalchemy](https://stackoverflow.com/questions/2268050/execute-sql-from-file-in-sqlalchemy).
https://docs.sqlalchemy.org/en/14/core/engines.html#sqlite

In [ ]:
# Generamos el motor de SQL con el nombre engine
path = os.path.join(DATA_PATH, 'motor_sql.sqlite3')
engine = create_engine(f'sqlite:///{path}', echo=True)

###Parte 2: 
Ingestar los datos provistos en 'https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/melb_data.csv' en una tabla y el dataset generado en clase con datos de airbnb y sus precios por codigo postal en otra.



####Seccion 1

Genero una tabla SQL **ven_melburn** con las columnas y los valores del data frame melb_df

In [ ]:
# Creamos una tabla de SQL ven_melburn y le cargo los datos del data frame melb_df
melb_df.to_sql('ven_melburn', con=engine, if_exists="replace")

In [ ]:
# Corremos un SQL de consulta como control rápido sobre la base
con = engine.connect()
rs = con.execute("SELECT count(*) FROM ven_melburn")
ven_df = pd.DataFrame(rs.fetchall())
ven_df.columns=rs.keys()
con.close()
ven_df

####Seccion 2

Levanto el archivo **airbnb_price_by_zipcode.csv** que generamos en clase con los datos de alquileres de Melburne. Analizamos el dataframe antes de pasarlo a una tabla SQL

In [ ]:
# Levantamos el dataset de airbnb
RAW_DATA_PATH = os.path.join(ROOT_PATH, 'data', 'raw')
airbnb_df = pd.read_csv(os.path.join(RAW_DATA_PATH, 'airbnb_price_by_zipcode.csv'))

In [ ]:
# Cuento cuantos registros tiene el data frame que cargue
len(airbnb_df)

In [ ]:
# Analizamos la información del data frame que subimos para verificar las columnas
# y nombres que tenemos disponibles
airbnb_df.info()

####Seccion 3

Generamos una tabla SQL **alq_melburn** con las columnas y los valores del data frame airbnb_df

In [ ]:
# Creo una tabla de SQL alq_melburn y le cargo los datos del data frame airbnb_df
airbnb_df.to_sql('alq_melburn', con=engine, if_exists="replace")

In [ ]:
# Corremos un SQL de consulta como control rapido sobre la base
con = engine.connect()
rs = con.execute("SELECT count(*) FROM alq_melburn")
alq_df = pd.DataFrame(rs.fetchall())
alq_df.columns=rs.keys()
con.close()
alq_df

###Parte 3:

Implementar consultas en SQL que respondan con la siguiente información:

    - cantidad de registros totales por ciudad.
    - cantidad de registros totales por barrio y ciudad.

 



####Seccion 1

Consultas sobre la tabla de Ventas en la ciudad de Melburn. 
https://dingoos.com/donde-vivir-en-melbourne/

In [ ]:
# Primero tenemos que analizar la información, para eso usamos
# el formato de data frame
melb_df.info()

In [ ]:
# Toda la informacion que tenemos en el data frame correspone a Melburne
# por lo tanto podemos utilizar la Region como ciudades que forman parte 
# de Melburne  
melb_df.Regionname.unique()

In [ ]:
# Corremos las sentencia SQL y presentamos la informacion usando 
# las tablas en formato pandas. Contamos los registros totales por
# Region considerando que las regiones son ciudades
con = engine.connect()
rs = con.execute("SELECT Regionname, count(*) as Registros \
                  FROM ven_melburn \
                  GROUP BY Regionname")
ven1_df = pd.DataFrame(rs.fetchall())
ven1_df.columns=rs.keys()
con.close()
ven1_df

In [ ]:
# Corremos las sentencia SQL y presntamos la información usando las tablas 
# en formato pandas. Contamos los registros totales por Region considerando
# que las regiones son ciudades y los Suburb son los Barrios
con = engine.connect()
rs = con.execute("SELECT Regionname AS Region, Suburb AS Barrio, round(AVG(price),2) AS Precio_Medio,\
                         count(*) AS Registros\
                  FROM ven_melburn \
                  GROUP BY Regionname, Suburb")
ven2_df = pd.DataFrame(rs.fetchall())
ven2_df.columns=rs.keys()
con.close()
ven2_df

###Parte 4
Combinar los datasets de ambas tablas ingestadas utilizando el comando JOIN de SQL  para obtener un resultado similar a lo realizado con Pandas en clase. 

**merged_sales_df = melb_df.merge(airbnb_price_by_zipcode, how='left',  left_on='Postcode', right_on='zipcode'**

In [ ]:
# Primero analizamos la información de los data frame que vamos a unir

melb_df.info()

In [ ]:
airbnb_df.info()

In [ ]:
# Despues de analizar ambos df, podemos ver que los campos de union de las tablas tienen el mismo Dtype
# entoncees ejecutamos tres sentencias SQL para registrar toda la informacion en una tabla !!! resultado !!!

# 1_ Sentencia de SQL para borrar una tabla si ya existe. 
conexion = engine.connect() 
borrar_tabla = conexion.execute("DROP TABLE IF EXISTS resultado") 
conexion.close()

#2_ Sentencia de SQL para crear una tabla desde un SELECT, a esta tabla la llamamos resultado. 
conexion=engine.connect() 
crear_tabla = conexion.execute("CREATE TABLE resultado AS \
                                             SELECT * FROM ven_melburn \
                                             LEFT JOIN alq_melburn ON \
                                             ven_melburn.Postcode = alq_melburn.zipcode")
conexion.close()

#3_ Sentencia de consulta de la información en la tabla nueva resumen. Tengo un resultado para mostrar
conexion=engine.connect()
resultado_ven = conexion.execute("SELECT * FROM resultado")
df_resultado_ven = pd.DataFrame(resultado_ven.fetchall())
# La sentencia abajo, coloca los nombres a las columnas en la tabla del df sino aparecen numeros
df_resultado_ven.columns = resultado_ven.keys()
conexion.close()
df_resultado_ven

Se realiza el JOIN de tablas manteniendo todos los datos de la tabla de Ventas que consideramos para nuestro análisis, a esta información lo que hacemos es agregar los precios de alquileres promediados en la zona que abarca el zipcode (LEFT JOIN). De esta forma tenemos información de las dos alternativas posibles considerando la zona donde se encuentra la propiedad. 

In [ ]:
DATA_PATH

In [ ]:
# La nueva tabla que combina ambas tablas se pasa a un archivo csv
conexion=engine.connect() 
db_df = pd.read_sql_query("SELECT * FROM resultado", conexion)
db_df.to_csv(os.path.join(DATA_PATH, 'resultado_join.csv'), index=False)

In [ ]:
# si se quiere ver el resultado:
# result = pd.read_csv(os.path.join(DATA_PATH, 'resultado_join.csv'))

## Ejercicio 2: 

### Parte 1

Seleccionar un subconjunto de columnas que les parezcan relevantes al problema de predicción del valor de la propiedad. Justificar las columnas seleccionadas y las que no lo fueron.
 - Eliminar los valores extremos que no sean relevantes para la predicción de valores de las propiedades.

In [ ]:
# Tomamos el data frame de Melburne con los datos de ventas de viviendas
melb_df.info()

####Sección 1

Seleccionamos algunas columnas del data frame que pueden servir para analizar los precios de las propiedades de Melburne.
Las variables seleccionadas son las que consideramos que de alguna manera pueden afectar al precio de la vivienda al momento de realizar la venta. Además pueden ser de utilidad cuando en pasos posteriores necesitemos otras variables para realizar un merge con la tabla de alquileres.

In [ ]:
# Selecciono algunas columnas del data frame
melb_cols = ['Suburb', 'Rooms', 'Type', 'Price', 'Method', 'SellerG', 'Date', 'Distance', 'Postcode', 'Bedroom2',
    'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Regionname']

In [ ]:
# Selecciono las columnas interesantes de las ventas realizadas en Melburn y las asigno
# al nuevo data frame melb_ven_df
melb_ven_df = melb_df [melb_cols]
melb_ven_df.head()

####Sección 2

Realizamos un histograma de la variable "Price" para elimiar los valores extermos que pueden afectar el análisis final de ventas.

In [ ]:
# Gráfico para determinar como se comporta la variable "Price"
# Puedemos comprobar que hay muchos precios por debajo de $250.000

seaborn.displot(melb_ven_df["Price"], aspect=5, kde=True, rug=False ) #kde=True realiza la grafica con la curva
plt.ticklabel_format(style='plain', axis='x') # Para evitar la notación científica en las etiquetas

In [ ]:
# Considerando el análisis anterior decidimos eliminar los valores extremos y achicar el tamaño del data frame 
# Creamos la funcion remove_outliers_by_IQR_method para eliminar del data frame, que le pasamos como dato, los valores
# maximos y minimos considerndo las dos (scale) desviaciones estandar a partir de los cuartiles de 25% (mínimo) 
# y el 75% (máximo)  
def remove_outliers_by_IQR_method(df, column, scale=2):
    q1= df[column].quantile(0.25)
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    
    lower_bound = q1 - (scale * iqr)
    upper_bound = q3 + (scale * iqr)
    
    df_without_outliers = df[(df[column] > lower_bound) 
                             & (df[column] < upper_bound)]
    
    return df_without_outliers

# Aplicamos la función que calcula los límites del rango que vamos a utulizar considerando el data frame con columnas resumido y le pasamos la 
# columna que deseamos quitar los outliers
melb_ven_df = remove_outliers_by_IQR_method(melb_ven_df, 'Price')

In [ ]:
seaborn.displot(melb_ven_df["Price"], aspect=5, kde=True, rug=False ) #kde=True me realiza la grafica con la curva

plt.ticklabel_format(style='plain', axis='x') # Para evitar la notación científica en las etiquetas

In [ ]:
# El data frame original contaba con 13580 al eliminar los extremos queda con un total de 13211 registros. 
# Consideramos que no es significativa la cantidad de registros eliminados por valores extremos
melb_ven_df.info()

### Parte 2

Agregar información adicional respectiva al entorno de una propiedad a partir del [conjunto de datos de AirBnB](https://www.kaggle.com/tylerx/melbourne-airbnb-open-data?select=cleansed_listings_dec18.csv) utilizado en el práctico.

In [ ]:
# https://en.wikipedia.org/wiki/List_of_Melbourne_suburbs
airbnb_df = pd.read_csv('https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/cleansed_listings_dec18.csv')

In [ ]:
airbnb_df.info()

#### Seccion 1

Seleccionar qué variables agregar y qué combinaciones aplicar a cada una. Por ejemplo, pueden utilizar solo la columna `price`, o aplicar múltiples transformaciones como la mediana o el mínimo.

In [ ]:
# Se seleccionaron del archivo general de los alquileres de airbnb los campos mas relevantes
# y que de alguna manera pueden ser considerados para realizar un merge con el data
# frame de ventas
airbnb_cols = ['description', 'neighborhood_overview',
    'street', 'neighborhood', 'city', 'suburb', 'state', 'zipcode',
    'price', 'weekly_price', 'monthly_price',
    'latitude', 'longitude']
 

In [ ]:
airbnb_df

In [ ]:
# Seleccionamos las columnas interesantes de los alquilres de Melburne y los asignamos
# al nuevo data frame melb_alq_df
melb_alq_df = airbnb_df[airbnb_cols]
melb_alq_df.info()

In [ ]:
# Analizamos los valores nulos que podemos encontrar en el dataframe de alquileres
melb_alq_df.isnull().sum()

In [ ]:
# Si hacemos el análisis de un campo para realizar la agrupación y calcular un precio promedio de alquiler
# seríaa el zipcode pero es de tipo object, por lo tanto en su contenido puede tener char o integer
melb_alq_df.zipcode.value_counts()

In [ ]:
# Corregimos el zipcode convirtiendolo a dato numerico
melb_alq_df['zipcode'] = pd.to_numeric(melb_alq_df.zipcode, errors='coerce')
melb_alq_df.zipcode.value_counts()

####Seccion 2.

Utilizar la variable zipcode para unir los conjuntos de datos. Sólo incluir los zipcodes que tengan una cantidad mínima de registros (a elección) como para que la información agregada sea relevante.


In [ ]:
# Como se puede ver la cantidad de viviendas que tienen alquiler semanal y mensual de airbnb
# es muy baja considerando las 22895 líneas del archivo de Melburne. De todas formas utilizamos el zipcode
# para agrupar y calcular un precio promedio de alquiler y para los alquileres
# semanales y mensuales
  
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'zipcode']

melb_alq_df_zipcode = melb_alq_df[relevant_cols].groupby('zipcode')\
   .agg(airbnb_record_count=('price', 'count'),
        airbnb_price_mean=('price', 'mean'),
        airbnb_weekly_price_mean=('weekly_price', 'mean'),
        airbnb_monthly_price_mean=('monthly_price', 'mean'))\
   .reset_index()

melb_alq_df_zipcode
        

In [ ]:
# Al hacer este group by vemos que tenemos muchas líneas con valores de un solo registro
# por lo tanto analizamos cuantos son para descartarlos

df_mask = melb_alq_df_zipcode['airbnb_record_count']<=1
filtered_df = melb_alq_df_zipcode[df_mask]
filtered_df

In [ ]:
# Tenemos 19 filas de 246, que estan por debajo de 1 registro con informacion de alquileres 
# conforman el 7,7% del total de filas, por lo tanto podemos elimiar esta informacion de la base
# antes de realizar la union con la tabla de ventas
df_mask = melb_alq_df_zipcode['airbnb_record_count']>=2
melb_alq_df_zipcode = melb_alq_df_zipcode[df_mask]

# Redondeamos todas las variables a 3 decimales
melb_alq_df_zipcode = round(melb_alq_df_zipcode,3)
melb_alq_df_zipcode

In [ ]:
# Realizamos la unión entre los archivos de ventas resumido y de alquileres corregido

merged_sales_df = melb_ven_df.merge(
    melb_alq_df_zipcode, how='left',
    left_on='Postcode', right_on='zipcode')
merged_sales_df

In [ ]:
plt.figure(figsize=(20,25))
seaborn.pairplot(data=merged_sales_df, y_vars=['airbnb_price_mean'],
                 x_vars=['Regionname'],
                 aspect=4, height=6)

#### Sección 3

Investigar al menos otras 2 variables que puedan servir para combinar los datos, y justificar si serían adecuadas o no. Pueden asumir que cuentan con la ayuda de anotadores expertos para encontrar equivalencias entre barrios o direcciones, o que cuentan con algoritmos para encontrar las n ubicaciones más cercanas a una propiedad a partir de sus coordenadas geográficas. **NO** es necesario que realicen la implementación.

Pueden leer otras columnas del conjunto de AirBnB además de las que están en `interesting_cols`, si les parecen relevantes.

In [ ]:
melb_ven_df.info()

In [ ]:
resumen_ven = melb_ven_df[['Suburb', 'Method']].groupby(['Suburb']).count(). reset_index()
resumen_ven

In [ ]:
melb_alq_df.info()


In [ ]:
resumen_alq = melb_alq_df[['suburb', 'city']].groupby(['suburb']).count(). reset_index()
resumen_alq

ALTERNATIVA 1: Viendo la composicion de los archivos podríamos tomar la variable Suburb como alternativa para realizar el **merge**. Pero antes debemos verificar que esta union de alquileres a la tabla de ventas sea valida. Para ello utilizamos la funcion **numpy.intersect1d** 

In [ ]:
intersection = numpy.intersect1d(resumen_alq.suburb.values, resumen_ven.Suburb.values, assume_unique=False)
print("Sales unique Suburb", len(resumen_ven.Suburb.unique()))# unificando todos los Suburb cuantos son unicos de la df resumen_ven
print("Airbnb unique Suburb", len(resumen_alq.suburb.unique()))# unificando todos los suburb cuantos son unicos de la df resumen_alq  
print("Common Suburb", len(intersection))# unificando todos los Suburb (barrios) cuantos tenemos iguales entre ambas intersecciones

In [ ]:
print('Records in Sales df with corresponding Suburb form Airbnb df',
      resumen_ven.Suburb.isin(intersection).sum() / len(resumen_ven)) # calculamos el % de Suburb que tenemos en resumen_ven y estan en resumne_alq

#print('Records in Airbnb df with corresponding Suburb form Sales df',
#      resumen_alq.suburb.isin(intersection).sum() / len(resumen_alq)) # calculamos el % de suburb que tenemos en resumen_alq y estan en resumen_ven

Como consideramos que el data frame principal es el de las Ventas en Melburne **melb_ven_df**, de nuestro analisis y verificacion que tenemos un 94% de los Suburb del data frame de Ventas estan en la tabla de precios de alquileres por lo tanto vale la pena realizar el **merge**

In [ ]:
relevant_cols = ['price', 'weekly_price', 'monthly_price', 'suburb']
melb_alq_df_suburb = melb_alq_df[relevant_cols].groupby('suburb')\
   .agg(airbnb_record_count=('price', 'count'),
        airbnb_price_mean=('price', 'mean'),
        airbnb_weekly_price_mean=('weekly_price', 'mean'),
        airbnb_monthly_price_mean=('monthly_price', 'mean')).reset_index()

melb_alq_df_suburb

In [ ]:
# Realizamos la union por el suburbio 
 
merged1_sales_df = melb_ven_df.merge(
    melb_alq_df_suburb, how='left',
    left_on='Suburb', right_on='suburb')
merged1_sales_df

In [ ]:
merged1_sales_df.isnull().sum()

Despues de realizar el merge creamos el data frame **merged1_sales_df**. Como vemos tenemos solo 124 filas de las 13211, un 1% del total del data frame de ventas, que quedaron los datos de presios de alquiler en blanco por no tener un registro para asociar el Suburb

ALTERNATIVA 2: Otra variable que podríamos usar para hacer el merge además de zipcode o suburb podría ser una nueva variable llamada cuadrante y que surge de dividir el mapa en cuadrantes usando las variables latitud y longitud de las tablas de ventas y alquileres. Usando un par ordenado con latitud, longitud se define el cuadrante al que pertenecen  y se le asignand un código/nombre a cada cuadrante en ambas tablas. Con esta nueva variable de localización que considera la cercanía geográfica en la localización de las propiedades podríamos hacer un merge entre ambas tablas (NO IMPLEMENTADO)

## Ejercicio 3:

Crear y guardar un nuevo conjunto de datos con todas las transformaciones realizadas anteriormente.
Hacer un reporte en pdf o txt parsa con el detalle de los pasos que se furon realizando

In [ ]:
merged_sales_df.to_csv(os.path.join(DATA_PATH, 'Ventas_join_zipcode.csv'), index=False)
merged1_sales_df.to_csv(os.path.join(DATA_PATH, 'Ventas_join_suburb.csv'), index=False)

En la estructura de la notebook se van detallando las secciones y los comentarios de cada celda, el detalle de los pasos que se fueron realizando para lograr los data frame de ventas uno realizando la relación con los alquileres usando el zipcode y el otro usando el suburb. 

Cuando finalicemos la parte 2 del entregable generaremos el archivo con todas las transformaciones para entregar junto con la notebook.  